In [1]:
import pickle
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import read_csv

In [2]:
filepath= "/Users/yanjia/Documents/extubation/processed_data/processedFrames.pkl"
OutputFrames = pickle.load(open(filepath, 'rb'))

In [3]:
key_out = OutputFrames.keys()

In [4]:
len(key_out)

7125

In [5]:
extubation_failure = read_csv('/Users/yanjia/Documents/extubation/extubation_failure/data/orginal_data.csv')

In [6]:
extubation_failure.head()

,Admission Weight (Kg),Admittype,Age,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,Arterial Blood Pressure systolic,Arterial CO2 Pressure,Arterial O2 pressure,Ethnicity,Gender,...,Peak Insp. Pressure,Plateau Pressure,Respiratory Rate,Respiratory Rate_spontaneous,Richmond-RAS Scale,SBT_value,Tidal Volume (observed),extubation_failure,hadm,ventilator_category
0,93.0,1,65.4,57.0,59.0,60.0,68.0,78.0,0,0,...,17,17.0,24.0,31.0,0.0,0,130.0,0.0,196611.0,5
1,113.0,1,63.9,54.0,73.0,120.0,40.0,80.0,0,0,...,11,23.0,24.0,25.0,-1.0,0,527.0,0.0,131076.0,5
2,123.5,1,39.6,53.0,66.0,105.0,49.0,138.0,1,0,...,10,16.0,19.0,20.0,1.0,0,578.0,0.0,182958.0,5
3,80.3,1,83.9,74.0,264.0,190.0,36.0,109.0,0,0,...,19,21.0,24.0,29.0,-1.0,0,567.0,1.0,180261.0,10
4,80.3,1,83.9,74.0,264.0,190.0,36.0,109.0,0,0,...,16,21.0,26.0,30.0,0.0,0,499.0,1.0,180261.0,10


In [7]:
one = extubation_failure[extubation_failure['extubation_failure'] == 1]

In [8]:
one.shape

(296, 27)

In [9]:
exfailure_hadm = one['hadm'].unique()

In [10]:
exfailure_hadm.shape

(249,)

In [11]:
admission_count = 0
rows = 0
rows_total = 0
df= DataFrame()
for key, chartFrame in OutputFrames.items():
    countFrame = chartFrame.drop(['firstICU', 'subject', 'Height (cm)', 'Non Invasive Blood Pressure mean', 'Non Invasive Blood Pressure systolic', 'Non Invasive Blood Pressure diastolic', 'Propofol','Fentanyl (Concentrate)', 'Midazolam (Versed)','Fentanyl','Dexmedetomidine (Precedex)','Morphine Sulfate','Hydromorphone (Dilaudid)','Lorazepam (Ativan)', 'SBT'], axis=1)
    if countFrame.isnull().values.any() == False:
        countFrame['Vented_1'] = countFrame['Vented'].shift(-1)
        df = df.append(countFrame, ignore_index=True)
        rows = len(countFrame.index)
        admission_count = admission_count +1
        rows_total = rows_total + rows

In [12]:
df.shape, admission_count, rows_total

((301964, 30), 2685, 301964)

In [13]:
df.head()

,timestamp,hadm,Admittype,Admdays,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,...,O2 Flow,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Vented,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1
0,2161-10-11 12:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,73.0,18.0,...,3.0,22.0,117.0,53.0,0.0,80.0,58.0,63.0,0.0,1.0
1,2161-10-11 13:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,60.5,17.5,...,3.0,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0.0,1.0
2,2161-10-11 14:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,63.0,18.0,...,3.0,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0.0,1.0
3,2161-10-11 15:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,68.0,18.0,...,3.0,22.0,117.0,53.0,1.0,95.0,64.0,75.0,0.0,1.0
4,2161-10-11 16:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,64.0,18.0,...,3.0,22.0,117.0,53.0,1.0,60.0,57.0,59.0,0.0,1.0


In [14]:
df = df.loc[~df['hadm'].isin(exfailure_hadm)]

In [15]:
corhot_df = df['hadm'].unique()

In [16]:
df.shape, len(corhot_df)

((223104, 30), 2436)

In [17]:
shortsupport_hadm = []
for i in corhot_df:
    if OutputFrames[i].shape[0]<8:
        shortsupport_hadm.append(i)

In [18]:
len(shortsupport_hadm)

137

In [19]:
df = df.loc[~df['hadm'].isin(shortsupport_hadm)]

In [20]:
len(df['hadm'].unique())

2299

In [21]:
df.head()

,timestamp,hadm,Admittype,Admdays,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,...,O2 Flow,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Vented,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1
0,2161-10-11 12:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,73.0,18.0,...,3.0,22.0,117.0,53.0,0.0,80.0,58.0,63.0,0.0,1.0
1,2161-10-11 13:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,60.5,17.5,...,3.0,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0.0,1.0
2,2161-10-11 14:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,63.0,18.0,...,3.0,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0.0,1.0
3,2161-10-11 15:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,68.0,18.0,...,3.0,22.0,117.0,53.0,1.0,95.0,64.0,75.0,0.0,1.0
4,2161-10-11 16:00:00,196611.0,1.0,8.18,0.0,0.0,65.4,93.0,64.0,18.0,...,3.0,22.0,117.0,53.0,1.0,60.0,57.0,59.0,0.0,1.0


In [22]:
ventilator = df['Ventilator Mode'].unique()

In [23]:
ventilator.shape

(23,)

In [24]:
np.sort(ventilator)

array([ 0.,  1.,  2.,  7., 10., 11., 12., 13., 14., 15., 16., 17., 26.,
       30., 45., 46., 47., 48., 49., 50., 51., 53., 71.])

In [25]:
ventilator_dictstring = {0.:'PRVC/AC', 1.:'CMV', 2.:'CMV/ASSIST', 6.:'SIMV', 7.:'SIMV/PSV', 10.:'CPAP', 11.:'CPAP/PSV', 
                         12.:'MMV', 13.:'MMV/PSV', 14.:'PCV+', 15.:'SYNCHRON MASTER', 16.:'SYNCHRON SLAVE', 
                         17.:'Apnea Ventilation', 26.:'APRV', 30.:'Standby', 45.: 'PCV+/PSV', 46.:'SIMV/AutoFlow', 
                         47.:'SIMV/PSV/AutoFlow', 48.:'CMV/AutoFlow', 49.:'CMV/ASSIST/AutoFlow', 50.:'MMV/AutoFlow',
                         51.:'MMV/PSV/AutoFlow', 53.:'CPAP/PPS', 71.:'PCV+Assist'}

In [26]:
ventilator_dict = {30.: 0,  0.:1,  1.:2, 2.:3, 7.:4, 10.:5, 11.:6, 12.:7, 13.:8, 14.:9, 15.:10, 16.:11,
       17.:12, 26.:13, 45.:14, 46.:15, 47.:16, 48.:17, 49.:18, 50.:19, 51.:20, 53.:21, 71.:22}

In [27]:
convert_dict = {'Admittype': int, 
                'Ethnicity': int,
                'Gender': int,
                'SBT_value': int,
                'O2 saturation pulseoxymetry': int,
                'Peak Insp. Pressure': int
                
               }

In [28]:
df['ventilator_category'] = df['Ventilator Mode'].apply(lambda x:ventilator_dict[x] )

In [29]:
df = df.astype(convert_dict) 

In [30]:
df = df.rename(columns={'Respiratory Rate (spontaneous)': 'Respiratory Rate_spontaneous'})

In [31]:
df['Admittype'].dtype

dtype('int64')

In [32]:
df = df.dropna()

In [33]:
df.shape

(220091, 31)

In [34]:
len(df['hadm'].unique())

2299

In [35]:
df.to_csv('data/orginal_data.csv', index= False)

In [36]:
# now split into train/validation/test sets
import random
#unique_ids = [*OutputFrames]
unique_ids = df['hadm'].unique()
random.shuffle(unique_ids)
train_sample = 0.8
val_sample = 0.1
test_sample = 0.1
train_num = int(len(unique_ids) * 0.8)
val_num = int(len(unique_ids)*0.1) + train_num
train_ids = unique_ids[:train_num]
val_ids = unique_ids[train_num:val_num]
test_ids = unique_ids[val_num:]

In [37]:
train_set = DataFrame()
train_set = df.loc[df['hadm'].isin(train_ids)]

val_set = DataFrame()
val_set = df.loc[df['hadm'].isin(val_ids)]

test_set = DataFrame()
test_set = df.loc[df['hadm'].isin(test_ids)]

In [38]:
len(unique_ids), len(train_ids), len(test_ids), len(val_ids), df.shape[0]

(2299, 1839, 231, 229, 220091)

In [39]:
train_set.head()

,timestamp,hadm,Admittype,Admdays,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,...,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Vented,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1,ventilator_category
0,2161-10-11 12:00:00,196611.0,1,8.18,0,0,65.4,93.0,73.0,18.0,...,22.0,117.0,53.0,0.0,80.0,58.0,63.0,0,1.0,3
1,2161-10-11 13:00:00,196611.0,1,8.18,0,0,65.4,93.0,60.5,17.5,...,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0,1.0,3
2,2161-10-11 14:00:00,196611.0,1,8.18,0,0,65.4,93.0,63.0,18.0,...,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0,1.0,3
3,2161-10-11 15:00:00,196611.0,1,8.18,0,0,65.4,93.0,68.0,18.0,...,22.0,117.0,53.0,1.0,95.0,64.0,75.0,0,1.0,3
4,2161-10-11 16:00:00,196611.0,1,8.18,0,0,65.4,93.0,64.0,18.0,...,22.0,117.0,53.0,1.0,60.0,57.0,59.0,0,1.0,3


In [40]:
train_set = train_set.drop(['timestamp', 'hadm', 'Admdays', 'Ventilator Mode', 'Vented'], axis=1)
val_set = val_set.drop(['timestamp', 'hadm', 'Admdays', 'Ventilator Mode', 'Vented'], axis=1)
test_set = test_set.drop(['timestamp', 'hadm', 'Admdays', 'Ventilator Mode', 'Vented'], axis=1)

In [41]:
train_set.head()

,Admittype,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,O2 saturation pulseoxymetry,Inspired O2 Fraction,PEEP set,...,O2 Flow,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1,ventilator_category
0,1,0,0,65.4,93.0,73.0,18.0,100,40.0,5.0,...,3.0,22.0,117.0,53.0,80.0,58.0,63.0,0,1.0,3
1,1,0,0,65.4,93.0,60.5,17.5,100,40.0,5.0,...,3.0,22.0,117.0,53.0,80.0,58.0,63.0,0,1.0,3
2,1,0,0,65.4,93.0,63.0,18.0,100,40.0,5.0,...,3.0,22.0,117.0,53.0,80.0,58.0,63.0,0,1.0,3
3,1,0,0,65.4,93.0,68.0,18.0,100,40.0,5.0,...,3.0,22.0,117.0,53.0,95.0,64.0,75.0,0,1.0,3
4,1,0,0,65.4,93.0,64.0,18.0,100,40.0,5.0,...,3.0,22.0,117.0,53.0,60.0,57.0,59.0,0,1.0,3


In [42]:
# now get the y lable #
trainy = train_set['Vented_1']
valy = val_set['Vented_1']
testy = test_set['Vented_1']

In [43]:
train_set = train_set.drop(['Vented_1'], axis=1)
val_set = val_set.drop(['Vented_1'], axis=1)
test_set = test_set.drop(['Vented_1'], axis=1)

In [44]:
trainy.to_csv('data/trainy.csv', index=False)
valy.to_csv('data/valy.csv', index=False)
testy.to_csv('data/testy.csv', index=False)

In [45]:
train_set.to_csv('data/train_set.csv',index = False)
val_set.to_csv('data/val_set.csv', index = False)
test_set.to_csv('data/test_set.csv', index = False)

In [46]:
# make the range between 0 and 1
column = list(train_set.columns.values)
for col in column:
    minimum = df[col].min()
    maximum = df[col].max()
    train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
    val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
    test_set[col] = (test_set[col] - minimum)/(maximum-minimum)

In [47]:
train_set.to_csv('data/train_set_scaled.csv', index=False)
val_set.to_csv('data/val_set_scaled.csv', index=False)
test_set.to_csv('data/test_set_scaled.csv', index=False)

In [48]:
train_set.head()

,Admittype,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,O2 saturation pulseoxymetry,Inspired O2 Fraction,PEEP set,...,Peak Insp. Pressure,O2 Flow,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,ventilator_category
0,1.0,0.0,0.0,0.165179,0.567901,0.289308,0.128571,1.0,0.4,0.333333,...,0.612903,0.12,0.53125,0.289604,0.58209,0.268908,0.214953,0.154070,0.5,0.136364
1,1.0,0.0,0.0,0.165179,0.567901,0.210692,0.125000,1.0,0.4,0.333333,...,0.612903,0.12,0.53125,0.289604,0.58209,0.268908,0.214953,0.154070,0.5,0.136364
2,1.0,0.0,0.0,0.165179,0.567901,0.226415,0.128571,1.0,0.4,0.333333,...,0.612903,0.12,0.53125,0.289604,0.58209,0.268908,0.214953,0.154070,0.5,0.136364
3,1.0,0.0,0.0,0.165179,0.567901,0.257862,0.128571,1.0,0.4,0.333333,...,0.612903,0.12,0.53125,0.289604,0.58209,0.331933,0.242991,0.188953,0.5,0.136364
4,1.0,0.0,0.0,0.165179,0.567901,0.232704,0.128571,1.0,0.4,0.333333,...,0.548387,0.12,0.53125,0.289604,0.58209,0.184874,0.210280,0.142442,0.5,0.136364


In [49]:
df[df['Ventilator Mode']==30]

,timestamp,hadm,Admittype,Admdays,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,...,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Vented,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1,ventilator_category
198,2182-07-05 04:00:00,131076.0,1,9.73,0,0,63.9,113.0,98.0,24.0,...,23.0,80.0,40.0,0.0,109.0,52.0,69.0,0,0.0,0
199,2182-07-05 05:00:00,131076.0,1,9.73,0,0,63.9,113.0,95.0,26.0,...,23.0,80.0,40.0,0.0,117.0,52.0,71.0,0,0.0,0
200,2182-07-05 06:00:00,131076.0,1,9.73,0,0,63.9,113.0,101.0,24.0,...,23.0,80.0,40.0,0.0,113.0,55.0,72.0,0,0.0,0
201,2182-07-05 07:00:00,131076.0,1,9.73,0,0,63.9,113.0,100.0,26.0,...,23.0,80.0,40.0,0.0,114.0,56.0,74.0,0,0.0,0
206,2152-01-14 15:00:00,182958.0,1,7.81,1,0,39.6,123.5,84.0,21.0,...,16.0,138.0,49.0,0.0,129.0,62.0,83.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299230,2199-01-02 12:00:00,180136.0,1,16.86,1,0,69.5,77.2,91.0,23.0,...,18.0,63.0,33.0,0.0,108.0,75.0,83.0,1,0.0,0
299813,2189-04-21 10:00:00,158373.0,1,14.58,0,1,53.7,66.7,96.0,8.0,...,15.0,110.0,39.0,1.0,122.0,65.0,89.0,0,0.0,0
299814,2189-04-21 11:00:00,158373.0,1,14.58,0,1,53.7,66.7,105.0,18.0,...,15.0,110.0,39.0,0.0,145.0,77.0,106.0,0,0.0,0
299815,2189-04-21 12:00:00,158373.0,1,14.58,0,1,53.7,66.7,103.0,20.0,...,15.0,110.0,39.0,0.0,164.0,84.0,119.0,0,0.0,0


In [54]:
df[df['Admittype']==1]

,timestamp,hadm,Admittype,Admdays,Ethnicity,Gender,Age,Admission Weight (Kg),Heart Rate,Respiratory Rate,...,Plateau Pressure,Arterial O2 pressure,Arterial CO2 Pressure,Vented,Arterial Blood Pressure systolic,Arterial Blood Pressure diastolic,Arterial Blood Pressure mean,SBT_value,Vented_1,ventilator_category
0,2161-10-11 12:00:00,196611.0,1,8.18,0,0,65.4,93.0,73.0,18.0,...,22.0,117.0,53.0,0.0,80.0,58.0,63.0,0,1.0,3
1,2161-10-11 13:00:00,196611.0,1,8.18,0,0,65.4,93.0,60.5,17.5,...,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0,1.0,3
2,2161-10-11 14:00:00,196611.0,1,8.18,0,0,65.4,93.0,63.0,18.0,...,22.0,117.0,53.0,1.0,80.0,58.0,63.0,0,1.0,3
3,2161-10-11 15:00:00,196611.0,1,8.18,0,0,65.4,93.0,68.0,18.0,...,22.0,117.0,53.0,1.0,95.0,64.0,75.0,0,1.0,3
4,2161-10-11 16:00:00,196611.0,1,8.18,0,0,65.4,93.0,64.0,18.0,...,22.0,117.0,53.0,1.0,60.0,57.0,59.0,0,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299858,2130-06-22 06:00:00,147441.0,1,15.12,0,0,48.6,65.0,50.0,14.0,...,17.0,220.0,34.0,1.0,118.0,55.0,79.0,1,1.0,6
299859,2130-06-22 07:00:00,147441.0,1,15.12,0,0,48.6,65.0,49.0,14.0,...,17.0,220.0,34.0,1.0,131.5,57.5,84.0,1,1.0,6
299860,2130-06-22 08:00:00,147441.0,1,15.12,0,0,48.6,65.0,58.0,16.0,...,17.0,220.0,34.0,1.0,151.5,66.0,97.5,1,1.0,6
299861,2130-06-22 09:00:00,147441.0,1,15.12,0,0,48.6,65.0,75.0,19.0,...,17.0,220.0,34.0,1.0,150.0,66.0,93.0,1,0.0,6


In [53]:
df.shape

(220091, 31)

In [55]:
df.columns

Index(['timestamp', 'hadm', 'Admittype', 'Admdays', 'Ethnicity', 'Gender',
       'Age', 'Admission Weight (Kg)', 'Heart Rate', 'Respiratory Rate',
       'O2 saturation pulseoxymetry', 'Inspired O2 Fraction', 'PEEP set',
       'Mean Airway Pressure', 'Ventilator Mode', 'Tidal Volume (observed)',
       'PH (Arterial)', 'Respiratory Rate_spontaneous', 'Richmond-RAS Scale',
       'Peak Insp. Pressure', 'O2 Flow', 'Plateau Pressure',
       'Arterial O2 pressure', 'Arterial CO2 Pressure', 'Vented',
       'Arterial Blood Pressure systolic', 'Arterial Blood Pressure diastolic',
       'Arterial Blood Pressure mean', 'SBT_value', 'Vented_1',
       'ventilator_category'],
      dtype='object')